In [1]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc,g1,g2,g3,NUC_MOD_OF,NUC_FRAC_CHARGE,PTR_FRAC_CHARGE
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x

In [3]:
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [4]:
ch4str

'C 0 0 0; H 0 0  2.043854241378284; H  0  1.9269642584472952 -0.6812847471260947; H 1.6688 -0.9634821292236474 -0.6812847471260947; H -1.6688 -0.9634821292236474 -0.6812847471260947'

In [5]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()

converged SCF energy = -40.2151804034982


-40.215180403498216

In [7]:
#ad1n,adn2=alc_deriv_grad_nuc(CH4,[1,0,0,0,0]),alc_differential_grad_nuc(CH4,[1,0,0,0,0])

In [16]:
cds=CH4.atom_coords()
for i in range(1,5):
    print(np.linalg.norm(cds[i]-cds[0]))

2.043854241378284
2.043854241378284
2.0438542413782836
2.0438542413782836


In [7]:
#np.save("ch4_opt_geom_bohr",CH4.atom_coords())

In [8]:
dl=.1
gs=[]
ge=[]
hs=[]
amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,0, 0, 0, 0],atom=ch4str,unit="Bohrs",\
           basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},verbose=0,charge=i*dl)
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e(),max_cycle=100)
    print(mf.mol.nelec)
    g=mf.Gradients()
    hs.append(mf.Hessian().hess())
    gs.append(g.grad())
    ge.append(g.grad_elec())
    amds.append(aaff(mf,[1,0,0,0,0]))

(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)


In [15]:
ch4_gs=np.asarray(gs)
ch4_ge=np.asarray(ge)
ch4_hs=np.asarray(hs)
ch4_amds=np.asarray(amds)

In [16]:
np.save("Ch4_grads_9pfd",ch4_gs),np.save("Ch4_g_elec_9pfd",ch4_ge),np.save("Ch4_hess_9pfd",ch4_hs)
np.save("Ch4_aderivs_9pfd",ch4_amds)

In [17]:
print("1 order finite diff. accuracy:: ",(ge[5]-ge[4])[1,2]/dl)
print("2 order finite diff. accuracy:: ",np.einsum('i,ijk->jk',fd1_2,ge)[1,2]/dl)
print("4 order finite diff. accuracy:: ",np.einsum('i,ijk->jk',fd1_4,(np.asarray(ge)))[1,2]/dl)
print("6 order finite diff. accuracy:: ",np.einsum('i,ijk->jk',fd1_6,(np.asarray(ge)))[1,2]/dl)
print("8 order finite diff. accuracy:: ",np.einsum('i,ijk->jk',fd1_8,(np.asarray(ge)))[1,2]/dl)
print("Analytical derivative        :: ",amds[4][1,2])

1 order finite diff. accuracy::  0.3163316347350942
2 order finite diff. accuracy::  0.3182340784545745
4 order finite diff. accuracy::  0.31830656870640905
6 order finite diff. accuracy::  0.31830665780282374
8 order finite diff. accuracy::  0.31830668755357144
Analytical derivative        ::  0.31830614343267605


In [18]:
for i in range(1,5):
    print(np.linalg.norm((ge1-ge2)[i]/2))

NameError: name 'ge1' is not defined

In [19]:
np.around(cds-cds[0],decimals=5)

array([[ 0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  2.04385],
       [ 0.     ,  1.92696, -0.68128],
       [ 1.6688 , -0.96348, -0.68128],
       [-1.6688 , -0.96348, -0.68128]])

In [51]:
NH4=FcM(fcs=[1,0,0,0,0],atom=ch4str,unit="bohr",\
          basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
NH4.nelectron=10

In [56]:
hfnh4=scf.RHF(NH4)
hfnh4.scf(dm0=hfnh4.init_guess_by_1e())

converged SCF energy = -56.5506646111421


-56.55066461114214

In [55]:
NH4.nelec

(5, 5)

In [57]:
gnh4,gnh4n=hfnh4.Gradients().grad()[1,2],hfnh4.Gradients().grad_nuc()[1,2]

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0000000000
1 H    -0.0000000000    -0.0000000000     0.0534595930
2 H    -0.0000000000     0.0504021876    -0.0178198643
3 H     0.0436495749    -0.0252010938    -0.0178198643
4 H    -0.0436495749    -0.0252010938    -0.0178198643
----------------------------------------------


In [13]:
gnh4n

-1.8955982396089313

In [15]:
mf.Gradients().grad_nuc()
ad1n

array([[ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17],
       [ 0.00000000e+00,  0.00000000e+00, -2.39386778e-01],
       [ 0.00000000e+00, -2.25696018e-01,  7.97955925e-02],
       [-1.95458485e-01,  1.12848009e-01,  7.97955925e-02],
       [ 1.95458485e-01,  1.12848009e-01,  7.97955925e-02]])

In [19]:
1/2.043854241378284**2

0.23938677763795033

In [65]:
hnh4=hfnh4.Hessian().hess()

In [66]:
gnh4,gnh4n,hnh4[1,1,2,2]

(0.0534595930096311, -1.8955982396089313, 0.30360466901718564)

In [58]:
BH4=FcM(fcs=[-1,0,0,0,0],atom=ch4str,unit="bohr",\
          basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
BH4.nelectron=10
hfbh4=scf.RHF(BH4)
hfbh4.scf(dm0=hfbh4.init_guess_by_1e())

converged SCF energy = -26.9391542012827


-26.939154201282676

In [59]:
gbh4,gbh4n=hfbh4.Gradients().grad()[1,2],hfbh4.Gradients().grad_nuc()[1,2]

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0000000000
1 H    -0.0000000000    -0.0000000000    -0.0900956221
2 H     0.0000000000    -0.0849429671     0.0300318740
3 H    -0.0735627674     0.0424714836     0.0300318740
4 H     0.0735627674     0.0424714836     0.0300318740
----------------------------------------------


In [61]:
hbh4=hfbh4.Hessian().hess()

In [64]:
gbh4,gbh4n,hbh4[1,1,2,2]

(-0.09009562209189137, -1.4168246843330305, 0.4029575636155609)

In [63]:
BH4.nelectron

10

In [6]:
alc_deriv_grad_nuc(CH4,[1,0,0,0,0]),alc_differential_grad_nuc(CH4,[1,0,0,0,0]),\
alc_deriv_grad_nuc(CH4,[-1,0,0,0,0]),alc_differential_grad_nuc(CH4,[-1,0,0,0,0])

(array([[ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17],
        [ 0.00000000e+00,  0.00000000e+00, -2.39386778e-01],
        [ 0.00000000e+00, -2.25696018e-01,  7.97955925e-02],
        [-1.95458485e-01,  1.12848009e-01,  7.97955925e-02],
        [ 1.95458485e-01,  1.12848009e-01,  7.97955925e-02]]),
 array([[ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17],
        [ 0.00000000e+00,  0.00000000e+00, -2.39386778e-01],
        [ 0.00000000e+00, -2.25696018e-01,  7.97955925e-02],
        [-1.95458485e-01,  1.12848009e-01,  7.97955925e-02],
        [ 1.95458485e-01,  1.12848009e-01,  7.97955925e-02]]),
 array([[ 0.00000000e+00,  1.11022302e-16,  5.55111512e-17],
        [ 0.00000000e+00,  0.00000000e+00,  2.39386778e-01],
        [ 0.00000000e+00,  2.25696018e-01, -7.97955925e-02],
        [ 1.95458485e-01, -1.12848009e-01, -7.97955925e-02],
        [-1.95458485e-01, -1.12848009e-01, -7.97955925e-02]]),
 array([[ 0.00000000e+00,  1.11022302e-16,  5.55111512e-17],
        [ 0.000000

In [ ]:
1/z